In [2]:
import pandas as pd
import json


def make_dict(df):
    user_dict={}
    item_dict={}
    user_list = list(set(df['reviewerID']))
    item_list = list(set(df['asin']))
    
    for i,j in enumerate(user_list):
        user_dict[j] = i
    for i,j in enumerate(item_list):
        item_dict[j] = i
    
    with open('d1_user_dict.json','w') as f:
        json.dump(user_dict, f)
    with open('d1_item_dict.json','w') as f:
        json.dump(item_dict, f)
    
    return user_dict, item_dict


def preprocess(df1, df2, user_dict, item_dict):
    df_test = df1.copy()
    df_train = df2.copy()
    
    df_test['reviewerID'] = df_test['reviewerID'].apply(lambda x:user_dict[x])
    df_test['asin'] = df_test['asin'].apply(lambda x:item_dict[x])
    df_train['reviewerID'] = df_train['reviewerID'].apply(lambda x:user_dict[x])
    df_train['asin'] = df_train['asin'].apply(lambda x:item_dict[x])
    
    df_test.to_csv('d1.test', index=False, header=False, sep=' ')
    df_train.to_csv('d1.train', index=False, header=False, sep=' ')


file='data.json'
df = pd.read_json(file)
df.drop(['reviewerName','helpful','reviewText','summary','unixReviewTime','reviewTime'], axis=1, inplace=True)
df = df.iloc[:800000]
df_test = df[df['reviewerID'].duplicated() & df['asin'].duplicated()].iloc[:400000:2]
df_train = pd.concat([df[df['reviewerID'].duplicated() & df['asin'].duplicated()].iloc[400000:],df[df['reviewerID'].duplicated() & df['asin'].duplicated()].iloc[1:400000:2] ,df[~df['reviewerID'].duplicated() | ~df['asin'].duplicated()]])
user_dict, item_dict = make_dict(df)
preprocess(df_test, df_train, user_dict, item_dict) 

